In [1]:
import os
import pickle
import numpy as np
import datetime
import json
from multiprocessing import Process

from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications import VGG16, MobileNetV2

from models import *
from utils import regression_stats
from img_utils import data_to_df, preprocess_images, set_gpu, set_cpu

2024-04-22 11:40:14.682598: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 11:40:14.682677: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 11:40:14.748645: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-22 11:40:14.955192: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 11:40:16.483811: W tensorflow/compiler/tf2

In [2]:
IMAGE_WIDTH: int = 448
IMAGE_HEIGHT: int = 448

# Load Data
train_1_path: str = "../nybolig-scrape/output/train/train_1"
train_2_path: str = "../nybolig-scrape/output/train/train_2"
valid_path: str = "../nybolig-scrape/output/valid"
test_path: str = "../nybolig-scrape/output/test"

train1_df, train2_df, valid_df, test_df = data_to_df(
    [train_1_path, train_2_path, valid_path, test_path], preprocess=True
)

# TODO: Would be better with this format
# train_images: np.array = preprocess_images(
#     train1_df["image_floorplan"], IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
# )
train_images: np.array = preprocess_images(
    train1_df, "image_floorplan", IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
)
train_prices: np.array = train1_df["price"].values

valid_images: np.array = preprocess_images(
    valid_df, "image_floorplan", IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
)
valid_prices: np.array = valid_df["price"].values

test_images: np.array = preprocess_images(
    test_df, "image_floorplan", IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
)
test_prices: np.array = test_df["price"].values

Processing ../nybolig-scrape/output/train/train_1: 100%|██████████| 311/311 [00:00<00:00, 1676643.37it/s]
Processing ../nybolig-scrape/output/train/train_2: 100%|██████████| 312/312 [00:00<00:00, 1869461.21it/s]
Preprocessing: 100%|██████████| 4/4 [00:00<00:00, 118.99it/s]


In [3]:
def save_model_and_evaluate(
    model: object,
    fit_history: object,
    test_images: np.array,
    test_prices: np.array,
    model_dir: str,
):
    # Save Model
    print("Saving Model...")
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model.save(f"{model_dir}/model")

    # Save Training History
    with open(f"{model_dir}/history", "wb") as file_pi:
        pickle.dump(fit_history.history, file_pi)

    # Evaluate Model
    print("Evaluating Model...")
    test_predictions = model.predict(test_images)
    r2, mae, percentage_error, mse = regression_stats(test_prices, test_predictions)

    try:
        feature_importance = model.feature_importance_
    except AttributeError:
        feature_importance = None

    # Load existing evaluation data
    evaluation_file_path = f"{model_dir}/evaluation.json"
    evaluation_data = {}
    if os.path.exists(evaluation_file_path):
        with open(evaluation_file_path, "r") as json_file:
            evaluation_data = json.load(json_file)

    # Add new evaluation data
    new_evaluation = {
        "Timestamp": str(datetime.datetime.now()),
        "R2": r2,
        "MAE": mae,
        "Percentage Error": percentage_error,
        "MSE": mse,
        "Feature Importances": str(feature_importance) if feature_importance else None,
    }
    evaluation_data[len(evaluation_data)] = new_evaluation

    # Save updated evaluation data
    with open(evaluation_file_path, "w") as json_file:
        json.dump(evaluation_data, json_file, indent=4)

    # Compute median evaluation values from all instances
    r2_values = [evaluation_data[key]["R2"] for key in evaluation_data]
    mae_values = [evaluation_data[key]["MAE"] for key in evaluation_data]
    percentage_error_values = [
        evaluation_data[key]["Percentage Error"] for key in evaluation_data
    ]
    mse_values = [evaluation_data[key]["MSE"] for key in evaluation_data]

    median_evaluation_data = {
        "R2": np.median(r2_values),
        "MAE": np.median(mae_values),
        "Percentage Error": np.median(percentage_error_values),
        "MSE": np.median(mse_values),
    }

    with open(f"{model_dir}/median_evaluation.json", "w") as json_file:
        json.dump(median_evaluation_data, json_file, indent=4)

    print("Model Evaluation:")
    print(new_evaluation)
    print("Median Evaluation:")
    print(median_evaluation_data)

    print("Done!")


def train_save_model(
    model_func: object,
    args: tuple,
    test_images: np.array,
    test_prices: np.array,
    model_dir: str,
    use_gpu: bool,
):
    if use_gpu:
        set_gpu()
    else:
        set_cpu()

    model, fit_history = model_func(*args)
    save_model_and_evaluate(model, fit_history, test_images, test_prices, model_dir)


def train_save_models(
    model_func: object,
    args: tuple,
    test_images: np.array,
    test_prices: np.array,
    model_dir: str,
    use_gpu: bool,
):
    if use_gpu:
        set_gpu()
    else:
        set_cpu()

    models, fit_histories = model_func(*args)
    for model_idx, (model, fit_history) in enumerate(zip(models, fit_histories)):
        save_model_and_evaluate(
            model, fit_history, test_images, test_prices, f"{model_dir}_{model_idx}"
        )

In [4]:
MODELS_PATH: str = "./models"
USE_GPU: bool = False

MODEL_NAME: str = "RF"
FUNCTION: object = RF
ARGS: tuple = (
    train_images,
    train_prices,
    valid_images,
    valid_prices,
)
p = Process(
    target=train_save_model,
    args=(FUNCTION, ARGS, test_images, test_prices, f"{MODELS_PATH}/{MODEL_NAME}", USE_GPU),
)

# MODEL_NAME: str = "VGG16"
# FUNCTION: object = CNN_model
# ARGS: tuple = (
#     VGG16,
#     True,
#     train_images,
#     train_prices,
#     valid_images,
#     valid_prices,
# )
# p = Process(
#     target=train_save_model,
#     args=(FUNCTION, ARGS, test_images, test_prices, f"{MODELS_PATH}/{MODEL_NAME}"),
# )

# MODEL_NAME: str = "N_CNN_MobileNetV2"
# FUNCTION: object = N_CNN_model
# ARGS: tuple = (
#     MobileNetV2,
#     train_images,
#     train_prices,
#     valid_images,
#     valid_prices,
#     3,
# )
# p = Process(
#     target=train_save_models,
#     args=(FUNCTION, ARGS, test_images, test_prices, f"{MODELS_PATH}/{MODEL_NAME}"),
# )

p.start()
p.join()

Setting CPU


Running RF on Features:
Fitting the model...


2024-04-22 11:41:15.441051: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 11:41:15.470336: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 11:41:15.470457: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
